<a href="https://colab.research.google.com/github/tejas-techstack/genai/blob/main/assignment_u2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q groq python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.7 MB/s eta 0:00:00


In [10]:
import os

from getpass import getpass
os.environ["GROQ_API_KEY"] = getpass("Enter your GROQ API Key: ")



Enter your GROQ API Key: ··········


In [24]:
from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY"))

BASE_MODEL = "llama-3.1-8b-instant"


In [25]:
MODEL_CONFIG = {
    "technical": {
        "system_prompt": """You are a senior technical support engineer.
Be precise, analytical, and code-focused.
Provide debugging steps and corrected code when necessary.
Avoid emotional language."""
    },
    "billing": {
        "system_prompt": """You are a customer billing specialist.
Be empathetic and professional.
Clearly explain policies, refunds, and charges clearly."""
    },
    "general": {
        "system_prompt": """You are a helpful general customer support assistant.
Respond politely and conversationally."""
    }
}


In [26]:
def route_prompt(user_input: str) -> str:
    router_prompt = f"""
Classify the following text into one of these categories:
[technical, billing, general]

Return ONLY the category name.

Text:
{user_input}
"""

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0,  # deterministic routing
        messages=[
            {"role": "system", "content": "You are an intent classifier."},
            {"role": "user", "content": router_prompt}
        ]
    )

    category = response.choices[0].message.content.strip().lower()

    if category not in MODEL_CONFIG:
        category = "general"

    return category


In [38]:
def process_request(user_input: str) -> str:
    category = route_prompt(user_input)
    print("The category is :", category)

    system_prompt = MODEL_CONFIG[category]["system_prompt"]

    response = client.chat.completions.create(
        model=BASE_MODEL,
        temperature=0.7,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
    )

    return response.choices[0].message.content.strip()

In [32]:
def fetch_bitcoin_price():
    return "Test value : this works!"

def enhanced_process_request(user_input: str) -> str:
    if "bitcoin" in user_input.lower() and "price" in user_input.lower():
        return f"The current price of Bitcoin is {fetch_bitcoin_price()}."

    return process_request(user_input)


In [43]:
while True:
    user_input = input("User: ")
    if user_input.lower() == "exit":
        break

    response = enhanced_process_request(user_input)
    print("Assistant:", response)


User: test
The category is : general
Assistant: It looks like you're just testing the waters, aren't you? Don't worry, I'm here to help if you need anything. What's on your mind?
User: it seems to work doesnt it
The category is : general
Assistant: It seems like we're having a good conversation so far. I'm happy to hear that everything seems to be working as expected. Is there something specific you were having issues with, or are you just checking in to make sure everything's okay?
User: the python code seems to work doesnt it
The category is : technical
Assistant: To determine if the code works as expected, I would need to see the code itself. Please provide the Python code you're referring to, and I'll be able to:

1. Review the code for any syntax errors or potential issues.
2. Analyze the code's functionality to ensure it meets your requirements.
3. Provide debugging steps or corrections if necessary.

Please paste the code, and I'll get started.
User: the billing doesnt seem to w

In [39]:
print(enhanced_process_request("Hello there!"))

The category is : general
Hello! It's nice to meet you. Is there something I can help you with today or would you like to just chat for a bit?


In [40]:
print(enhanced_process_request("My Python script throws an IndexError on line 5."))

The category is : technical
To help you further, I'll need more information about your script, such as:

1. The content of line 5.
2. The surrounding code.
3. The full error message (including any error codes or tracebacks).
4. The Python version you're using.

Please provide these details, and I'll guide you through the debugging process.

In general, an `IndexError` occurs when you're trying to access an element in a list or other sequence that doesn't exist. This often happens when the index you're using is out of range.

Here's a basic example of how we can debug an `IndexError`:

```python
# Example code
my_list = [1, 2, 3]
index = 5

try:
    print(my_list[index])
except IndexError as e:
    print(f"Error: {e}")
```

In this example, the error message will be "Error: list index out of range."

If you're using Python, you can also use a debugger or a tool like PDB to step through your code and identify the issue.

Please provide more information, and I'll help you debug your scrip

In [41]:
print(enhanced_process_request("I was charged twice for my subscription this month."))

The category is : billing
I'm so sorry to hear that you've been charged twice for your subscription this month. I'm here to help you resolve the issue as quickly and efficiently as possible.

To begin, I'd like to thank you for bringing this to my attention. I'm going to check on the status of your account and see what might have caused the duplicate charge. Can you please provide me with your account information, including your name, address, and the last four digits of your payment method? This will help me locate your account and look into the issue.

In the meantime, I'd like to assure you that we take situations like this very seriously. We strive to provide accurate and timely billing to all our customers, and I'm committed to making things right for you.

Once I've located your account, I'll check to see if the duplicate charge has been processed yet. If it has, I'll work on getting a refund issued to you as soon as possible. If it hasn't, I can stop the charge from processing a

In [42]:
print(enhanced_process_request("What is the current price of Bitcoin?"))

The current price of Bitcoin is Test value : this works!.
